<a href="https://colab.research.google.com/github/RounabhSahu/Blogging-App/blob/master/Attention_U_Net_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import io
import random
import nibabel
import numpy as np
from glob import glob
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint

from nibabel import load
import matplotlib.pyplot as plt
from keras.utils import Sequence
from IPython.display import Image, display
from skimage.exposure import rescale_intensity
from skimage.segmentation import mark_boundaries
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual,FloatRangeSlider,FloatSlider
import ipywidgets as widgets

from google.colab import drive
from scipy.ndimage import zoom
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Set your data path
data_path = '/content/drive/MyDrive/HYPODENSITY-DATA/'

In [6]:
all_files = sorted(glob(os.path.join(data_path, '*')))
print(all_files)

['/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg006', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg013', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg014', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg021', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg022', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg025', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg043', '/content/drive/MyDrive/HYPODENSITY-DATA/ProxmedImg331']


In [9]:
class BrainHypoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_filenames, mask_filenames, batch_size, image_size):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.batch_size = batch_size
        self.image_size = image_size
        self.length = int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.mask_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]

        x = np.zeros((self.batch_size, *self.image_size, 1))
        y = np.zeros((self.batch_size, *self.image_size, 1))

        for i, (image_filename, mask_filename) in enumerate(zip(batch_x, batch_y)):
            image = nib.load(image_filename)
            mask = nib.load(mask_filename)
            # get the data from the image object
            image_data = image.get_fdata()
            mask_data = mask.get_fdata()
            # get random slice from the volumes
            slice_index = np.random.randint(0, image_data.shape[2] - 1)
            x[i, :, :, 0] = image_data[:, :, slice_index]
            y[i, :, :, 0] = (mask_data[:, :, slice_index] > 0).astype(np.float32)  # Assuming mask is binary

        return x, y

# Get the list of train images and masks
train_images = sorted(glob(os.path.join(data_path, '*', '*_NCCT.nii.gz')))
train_masks = sorted(glob(os.path.join(data_path, '*', '*_ROI.nii.gz')))

# Set batch size and image size
batch_size = 1
image_size = (512, 512)

# Create data generators
train_generator = BrainHypoDataGenerator(train_images[:10], train_masks[:10], batch_size, image_size)
val_generator = BrainHypoDataGenerator(train_images[10:], train_masks[10:], batch_size, image_size)

In [23]:
from tensorflow.keras.layers import Input,Conv1D,Conv2D, Conv3D, UpSampling3D, concatenate, Softmax,UpSampling2D
from tensorflow.keras.models import Model

def attention_block(x, g, inter_channel):
    theta = Conv2D(inter_channel, (1, 1), activation='relu', padding='same')(x)
    phi = Conv2D(inter_channel, (1, 1), activation='relu', padding='same')(x/2)

    # Upsample phi to match the dimensions of theta
    phi = Conv2D(inter_channel, (1, 1), activation='relu', padding='same')(phi)

    attn = Softmax(axis=-1)(theta * phi)
    y = Conv2D(inter_channel, (1, 1), activation='relu', padding='same')(attn * x)
    return y


def convolution_block(x, filters, kernel_size, strides=(1, 1), activation='relu', padding='same'):
    x = Conv2D(filters, kernel_size, strides=strides, activation=activation, padding=padding)(x)
    return x

def attention_unet_2d(input_shape, filters=64, attention_inter_channel=32):
    inputs = Input(input_shape)

    # Encoder
    conv1 = convolution_block(inputs, filters, (3, 3))
    # No pooling in the encoder
    conv2 = convolution_block(conv1, filters * 2, (3, 3))

    # Attention Block
    attention_g = UpSampling2D(size=(2, 2))(conv2)  # Upsample for matching dimensions
    attention_block_2 = attention_block(conv2, attention_g, attention_inter_channel)

    # Decoder
    up1 = UpSampling2D(size=(2, 2))(conv2)
    concat1 = concatenate([up1, attention_block_2], axis=-1)
    conv3 = convolution_block(concat1, filters, (3, 3))

    up2 = UpSampling2D(size=(2, 2))(conv3)
    concat2 = concatenate([up2, conv1], axis=-1)
    conv4 = convolution_block(concat2, filters, (3, 3))

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv4)

    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model and specify loss, optimizer, and metrics
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Create the model with input shape
input_shape = (*image_size, 1)
model = attention_unet_2d(input_shape)

# Train the model with your dataset
model.fit(train_generator, epochs=epochs, validation_data=val_generator)

ValueError: ignored

In [30]:
test_loss, test_accuracy = model.evaluate(test_generator)

# Print the test loss and accuracy
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

8/8 [==============================] - 13s 1s/step - loss: 0.0553 - accuracy: 0.9896
Test Loss: 0.05528793856501579
Test Accuracy: 0.9895777702331543
